<a href="https://colab.research.google.com/github/KenzaxTazi/Agri-Risk/blob/master/Model_exploration_Raghul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm==4.42.1


In [2]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o



Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 126.2MB 78kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.3-py2.py3-none-any.whl size=126303006 sha256=ac57133de6f9189347a878da649946a4214afa84321bcd062d9d36808ceab891
  Stored in directory: /root/.cache/pip/wheels/fa/7c/ce/95ae52b4d3f1b14a27c3c961c1f94635aee841ab1eec3aeeca
Successfully built h2o


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import xarray as xr
import numpy as np


In [0]:
import os, sys

path = '/content/drive/My Drive/Team Plants/data'
os.chdir(path)

# Import full dataset # 

Here, we import the below dataset

In [0]:
dataset = pd.read_csv("head_of_soils_recommendations_MGM.csv")

In [0]:
dataset

,Unnamed: 0,Unnamed: 0.1,alloc_key,x,y,iso3,maize_a_2010,maize_a_2005,nearest_lat,nearest_lon,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,CDD-Q1-2005,CDD-Q2-2005,CDD-Q3-2005,CDD-Q4-2005,CFD-Q1-2010,CFD-Q2-2010,CFD-Q3-2010,CFD-Q4-2010,CFD-Q1-2005,CFD-Q2-2005,CFD-Q3-2005,CFD-Q4-2005,CWD-Q1-2010,CWD-Q2-2010,CWD-Q3-2010,CWD-Q4-2010,CWD-Q1-2005,CWD-Q2-2005,CWD-Q3-2005,CWD-Q4-2005,WW-Q1-2010,WW-Q2-2010,WW-Q3-2010,WW-Q4-2010,WW-Q1-2005,WW-Q2-2005,...,FD-04-05-2010,FD-04-05-2005,FD-05-05-2010,FD-05-05-2005,FD-06-05-2010,FD-06-05-2005,FD-07-05-2010,FD-07-05-2005,FD-08-05-2010,FD-08-05-2005,FD-09-05-2010,FD-09-05-2005,FD-10-05-2010,FD-10-05-2005,FD-11-05-2010,FD-11-05-2005,FD-12-05-2010,FD-12-05-2005,FD-01-15-2010,FD-01-15-2005,FD-02-15-2010,FD-02-15-2005,FD-03-15-2010,FD-03-15-2005,FD-04-15-2010,FD-04-15-2005,FD-05-15-2010,FD-05-15-2005,FD-06-15-2010,FD-06-15-2005,FD-07-15-2010,FD-07-15-2005,FD-08-15-2010,FD-08-15-2005,FD-09-15-2010,FD-09-15-2005,FD-10-15-2010,FD-10-15-2005,soil_types,climate_zones
0,0,0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,53.75,123.25,27.0,29.0,9.0,17.0,57.0,33.0,14.0,36.0,90.0,61.0,0.0,65.0,90.0,65.0,1.0,64.0,3.0,5.0,6.0,4.0,2.0,3.0,7.0,5.0,3.0,0.0,1.0,2.0,0.0,0.0,...,10.0,10.0,7.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,0.0,10.0,10.0,40,9
1,1,3,4393629,122.375000,53.458333,CHN,3119.3,2028.5,53.25,122.25,27.0,22.0,9.0,17.0,34.0,49.0,10.0,13.0,90.0,61.0,1.0,65.0,90.0,65.0,1.0,64.0,3.0,5.0,4.0,4.0,1.0,2.0,8.0,3.0,2.0,0.0,2.0,1.0,0.0,0.0,...,10.0,10.0,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,10.0,10.0,40,9
2,2,8,4403648,123.958333,53.375000,CHN,3230.1,2337.9,53.25,123.75,27.0,22.0,9.0,16.0,48.0,49.0,11.0,11.0,90.0,61.0,1.0,65.0,90.0,65.0,2.0,61.0,3.0,5.0,4.0,3.0,2.0,3.0,5.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,...,10.0,10.0,8.0,7.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,10.0,10.0,40,6
3,3,9,4403649,124.041667,53.375000,CHN,3119.3,2010.2,53.25,124.25,20.0,22.0,9.0,17.0,48.0,49.0,11.0,37.0,90.0,61.0,0.0,65.0,90.0,65.0,1.0,61.0,3.0,5.0,4.0,3.0,2.0,3.0,5.0,3.0,1.0,1.0,3.0,0.0,0.0,0.0,...,10.0,10.0,8.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,10.0,10.0,40,6
4,4,10,4413637,123.041667,53.291667,CHN,3918.1,2819.2,53.25,123.25,27.0,29.0,9.0,16.0,27.0,49.0,8.0,14.0,90.0,61.0,1.0,73.0,90.0,65.0,1.0,61.0,3.0,5.0,4.0,4.0,2.0,2.0,7.0,3.0,3.0,0.0,1.0,1.0,0.0,0.0,...,10.0,10.0,8.0,8.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,4.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,10.0,10.0,40,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427053,427053,832697,7660984,-98.041667,26.208333,USA,6148.3,6278.7,26.25,-97.25,16.0,23.0,33.0,16.0,12.0,48.0,24.0,19.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,11.0,13.0,10.0,3.0,2.0,6.0,7.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76,-997
427054,427054,832720,7670981,-98.291667,26.125000,USA,5713.1,5832.9,26.25,-97.25,16.0,23.0,33.0,16.0,12.0,48.0,24.0,19.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,11.0,13.0,10.0,3.0,2.0,6.0,7.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76,-997
427055,427055,832721,

In [0]:
data = dataset.copy()

## Data processing ## 

We now use Ed's script to process the data somewhat. Here we exclude the 10 day features.

In [9]:
# Create a custom data set adding in the features you want. It reads an already computed 
# dataset but you can then remove features to see the effect on score.
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random

random_seed = 42
#random.seed(a=random_seed)
#np.random_seed(random_seed)

read_file_path = f"head_of_soils_recommendations_MGM.csv"

# Only read a sample fraction of the data, speeds things up, but you'll get worse performance.
# Use 1.0 as the sample fraction to get the full data set
sample_fraction = 0.9
data = pd.read_csv(read_file_path, skiprows=lambda i: i>0 and random.random() > sample_fraction)

limit_change_in_yield = True
limit = 0.5

include_coordinates = True
include_2005_maize = True
include_seasonal = True
include_ten_day = False
include_2005_climate = True
include_soil_type = True
include_growing_zones = True
include_climate_zones = True

# Remove datapoints where yield changes by more than 50%. This is unlikely to be
# due to changes in climate change.

if limit_change_in_yield:
    data = data[abs(data['maize_a_2010'] - data['maize_a_2005']) / data['maize_a_2005'] < limit]

# Choose a subset of your features! (must already exist in original file)
ten_day_features = [
    'BEDD',
    'R20mm',
    'DTR'
]

seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]

features = []

if include_growing_zones:
    growing_zones = [
        'Inland water bodies',
        'Subtropics - summer rainfall',
        'Subtropics - winter rainfall',
        'Temperature - continental',
        'Temperature - oceanic',
        'Temperature - subcontinental',
        'Tropics'
    ]
    
    
if include_soil_type:
    features = features + ['soil_types']

        
if include_climate_zones:
    features = features + ['climate_zones']

if include_2005_climate:
    years = ['2010', '2005']
else:
    years = ['2010']

if include_seasonal:
    seasons = ['Q1', 'Q2', 'Q3', 'Q4']
    years = ['2010', '2005']
    for feature in seasonal_features:
        for season in seasons:
            for year in years:
                features.append(f'{feature}-{season}-{year}')

if include_ten_day:
    days = ['05', '15', '25']
    years = ['2010', '2005']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    for feature in ten_day_features:
        for day in days:
            for month in months:
                for year in years:
                    features.append(f'{feature}-{month}-{day}-{year}')

if include_coordinates:
    coordinates = [
        'x',
        'y'
    ]
    features = features + coordinates
    
if include_2005_maize:
    features = features + ['maize_a_2005']

target = 'maize_a_2010'
data = data.dropna()

X = data[features]
X.drop('x', axis=1, inplace=True) #drop x but keep y so we can tell something about the hemispheres and seasons


y = data[[target]]
train_set_x, test_set_x, train_set_y, test_set_y = train_test_split(X, y, test_size=0.2, random_state=random_seed)
del X
del y
del data

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
# drop features we don't want

#dataset = dataset.drop(['x','y','iso3_2005','Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'delta_maiz', 'maiz_percent_change'], axis=1)

In [0]:
train_set_x

,soil_types,climate_zones,CDD-Q1-2010,CDD-Q1-2005,CDD-Q2-2010,CDD-Q2-2005,CDD-Q3-2010,CDD-Q3-2005,CDD-Q4-2010,CDD-Q4-2005,CFD-Q1-2010,CFD-Q1-2005,CFD-Q2-2010,CFD-Q2-2005,CFD-Q3-2010,CFD-Q3-2005,CFD-Q4-2010,CFD-Q4-2005,CWD-Q1-2010,CWD-Q1-2005,CWD-Q2-2010,CWD-Q2-2005,CWD-Q3-2010,CWD-Q3-2005,CWD-Q4-2010,CWD-Q4-2005,WW-Q1-2010,WW-Q1-2005,WW-Q2-2010,WW-Q2-2005,WW-Q3-2010,WW-Q3-2005,WW-Q4-2010,WW-Q4-2005,WSDI-Q1-2010,WSDI-Q1-2005,WSDI-Q2-2010,WSDI-Q2-2005,WSDI-Q3-2010,WSDI-Q3-2005,WSDI-Q4-2010,WSDI-Q4-2005,CSDI-Q1-2010,CSDI-Q1-2005,CSDI-Q2-2010,CSDI-Q2-2005,CSDI-Q3-2010,CSDI-Q3-2005,CSDI-Q4-2010,CSDI-Q4-2005,y,maize_a_2005
56592,40,1,57.0,90.0,92.0,59.0,10.0,14.0,24.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,49.0,11.0,15.0,19.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,27.0,0.0,0.0,6.0,8.0,0.0,0.0,6.0,0.0,6.0,0.0,0.0,0.0,6.0,24.208333,943.0
223807,91,-997,7.0,17.0,9.0,9.0,16.0,29.0,12.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,6.0,4.0,7.0,4.0,6.0,3.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-28.291667,1740.2
188591,25,1,1.0,1.0,15.0,9.0,43.0,51.0,25.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0,55.0,32.0,40.0,2.0,1.0,45.0,27.0,15.0,6.0,3.0,5.0,1.0,0.0,5.0,7.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.291667,3817.2
176435,7,1,9.0,5.0,2.0,5.0,4.0,13.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,19.0,30.0,52.0,22.0,7.0,22.0,28.0,1.0,0.0,13.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.541667,1922.8
64412,4,1,22.0,39.0,10.0,8.0,1.0,1.0,19.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,11.0,11.0,50.0,60.0,32.0,21.0,0.0,0.0,0.0,0.0,21.0,8.0,9.0,2.0,9.0,8.0,8.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,9.0,0.0,22.208333,2309.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149615,95,5,6.0,17.0,8.0,16.0,26.0,30.0,25.0,18.0,25.0,79.0,11.0,18.0,0.0,0.0,7.0,11.0,11.0,3.0,3.0,6.0,7.0,2.0,5.0,6.0,7.0,4.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.291667,5771.2
332256,40,1,5.0,6.0,31.0,34.0,63.0,92.0,60.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,16.0,7.0,4.0,1.0,0.0,9.0,4.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-17.375000,1010.5
164359,84,1,38.0,44.0,39.0,21.0,5.0,9.0,27.0,33.0,6.0,7.0,1.0,0.0,0.0,0.0,5.0,2.0,3.0,1.0,4.0,6.0,14.0,21.0,11.0,11.0,0.0,0.0,0.0,0.0,4.0,9.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.375000,4132.8
184249,50,1,13.0,27.0,12.0,14.0,49.0,54.0,50.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,4.0,10.0,14.0,4.0,2.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.208333,875.7


In [0]:
#dataset.dropna(inplace=True)

### Create train test split ### 

In [0]:
#from sklearn.model_selection import train_test_split

#train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

#train_set_x = train_set.drop("maiz_a_2010",axis=1)
#train_set_y = train_set["maiz_a_2010"]

#test_set_x = test_set.drop("maiz_a_2010",axis=1)
#test_set_y = test_set["maiz_a_2010"]

### One-hot encode categorical features ###


In [0]:
categorical_variables = ["climate_zones",
                         "soil_types"
                       
                         ]

In [0]:
categories=[]

for i in categorical_variables:
  a = list(dataset[i].unique())
  categories.append(a)


In [0]:
df_num = train_set_x.copy()

for i in categorical_variables:
  df_num.drop(i, axis=1, inplace=True)

Below pipeline didn't work one time for the onehotencoding so it's been commented out and replaced.

In [0]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class DataFrameSelector(BaseEstimator, TransformerMixin):
#   def __init__(self,attribute_names):
#     self.attribute_names = attribute_names
#   def fit(self, X, y=None):
#     return self
#   def transform(self, X):
#     return X[self.attribute_names].values

# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.preprocessing import OneHotEncoder

# num_attributes = list(df_num)
# cat_attribs = categorical_variables

# num_pipeline = Pipeline([
#                          ('selector', DataFrameSelector(num_attributes)),
# ])

# cat_pipeline = Pipeline([
#                          ('selector', DataFrameSelector(cat_attribs)),
#                          ('one_hot_encoder', OneHotEncoder(categories = categories)),
                         
# ])

# full_pipeline = FeatureUnion(transformer_list=[
#                   ("num_pipeline",num_pipeline),
#                   ("cat_pipeline", cat_pipeline)
# ])

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
                        ("selector", OneHotEncoder(), [0,1])], 
                        remainder = 'passthrough')

#0 and 1 refer to the categorical column names - these need to be changed so they are selected automatically and not manually as done here

In [0]:
train_set_x_prepared = ct.fit_transform(train_set_x)


In [0]:
train_set_x_prepared.shape

(243624, 154)

In [0]:
test_set_x_prepared = ct.transform(test_set_x)


In [0]:
pd.DataFrame(train_set_x_prepared)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,13.0,9.0,89.0,92.0,16.0,12.0,0.0,0.0,1.0,0.0,7.0,11.0,2.0,3.0,15.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.208333,3941.3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,14.0,10.0,8.0,6.0,6.0,8.0,7.0,8.0,4.0,0.0,1.0,5.0,3.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-28.458333,5159.6
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,28.0,0.0,0.0,17.0,12.0,3.0,5.0,3.0,4.0,6.0,7.0,4.0,3.0,2.0,1.0,1.0,1.0,8.0,1.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.375000,10113.1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,44.0,8.0,34.0,41.0,49.0,16.0,46.0,64.0,1.0,0.0,6.0,2.0,1.0,2.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.625000,2111.6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,2.0,79.0,62.0,10.0,18.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,2.0,0.0,0.0,15.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.125000,1788.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,4.0,0.0,0.0,1.0,1.0,9.0,4.0,7.0,4.0,5.0,4.0,3.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,39.208333,11711.3
243620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.0,21.0,17.0,22.0,0.0,0.0,18.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,12.0,6.0,0.0,8.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,-17.541667,1373.9
243621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,11.0,24.0,19.0,19.0,23.0,24.0,24.0,5.0,0.0,4.0,4.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,19.375000,1809.0
243622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,26.0,23.0,30.0,2.0,1.0,9.0,6.0,0.0,5.0,2.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.291667,1596.8


### Rough feature engineering ### 

1. Exclude features with low variance

In [0]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.2)
train_set_x_reduced = selector.fit_transform(train_set_x_prepared)
test_set_x_reduced = selector.transform(test_set_x_prepared)

In [0]:
train_set_x_reduced.shape

(243695, 51)

In [0]:
selector.get_support() #these are the features which are kept ... need to work out which ones they are

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

2. Or/and train a simple extra trees model and select most important features

In [0]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

simple_model = ExtraTreesRegressor(n_jobs=-1, verbose=1, n_estimators=10)
simple_model = simple_model.fit(train_set_x_prepared, train_set_y)






/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


In [0]:
simple_model.score(train_set_x_prepared, train_set_y) # we are able to fit to the training set at least

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.2s finished


0.99981701893788

In [0]:
simple_model.score(test_set_x_prepared, test_set_y) 

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


0.9668285578750894

The below selects features based on feature importance but only gives 4 features when selecting based on mean, so I've not done it here.

In [0]:
# selector = SelectFromModel(simple_model, prefit=True)
# train_set_x_reduced = selector.transform(train_set_x_prepared)
# test_set_x_reduced = selector.transform(test_set_x_prepared)

#### Feature engineering without one-hot encoding ####

In [0]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.2)
train_set_x_reduced_2 = selector.fit_transform(train_set_x)
test_set_x_reduced_2 = selector.transform(test_set_x)

In [0]:
train_set_x_reduced_2.shape

(243618, 52)

This doesn't decrease number of features. 52 is actually fine. So I don't keep the transformed features as it messes up with the indices.. (I'm sure I can sort it out but not for now)

# Model exploration # 

## Which models can at least fit the training data? ##

### Random Forests ###


For this example we can run without doing the feature selection above.

In [0]:
from sklearn.ensemble import RandomForestRegressor

rnd_reg = RandomForestRegressor(n_estimators=15, n_jobs=-1)
rnd_reg.fit(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=15, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

Can we overfit to the training set? Does the model have enough sophisitication to overfit to the trainin set...

In [0]:
rnd_reg.score(train_set_x_prepared,train_set_y)

0.9945966246464768

In [0]:
rnd_reg.score(test_set_x_prepared,test_set_y)

0.9686268838624702

It could overfit if I train with enough max leaf nodes.




#### H2o implementation ####

In [0]:
import h2o
h2o.init(nthreads = -1, max_mem_size = 12)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyn0lo2_y
  JVM stdout: /tmp/tmpyn0lo2_y/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpyn0lo2_y/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_unknownUser_1feeo2
H2O cluster total nodes:,1
H2O cluster free memory:,12 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
train = pd.DataFrame(train_set_x).join(train_set_y)
train = h2o.H2OFrame(train)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
test = pd.DataFrame(test_set_x).join(test_set_y)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:

x = train.col_names[:-1]    
y = train.col_names[-1]  

In [0]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1, categorical_encoding='enum', max_depth=200)
rf_fit1.train(x=x, y=y, training_frame=train, validation_frame=test)


drf Model Build progress: |███████████████████████████████████████████████| 100%


In [0]:
rf_fit1.r2()


0.972332218086161

In [0]:
rf_fit1.r2(valid=True)

0.9736595078639356

In [0]:
test = pd.DataFrame(test_set_x).join(test_set_y)
test = h2o.H2OFrame(test)

rf_fit1.model_performance(test_data=test)

Parse progress: |█████████████████████████████████████████████████████████| 100%

ModelMetricsRegression: drf
** Reported on test data. **

MSE: 309762.0713281609
RMSE: 556.5627290145836
MAE: 312.822390934289
RMSLE: 0.14385844917400437
Mean Residual Deviance: 309762.0713281609


### Extra trees ###


In [0]:
from sklearn.ensemble import ExtraTreesRegressor

extra_trees = ExtraTreesRegressor(n_jobs=-1, verbose=1, n_estimators=10)
extra_trees = extra_trees.fit(train_set_x_prepared, train_set_y)

extra_trees.score(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.3min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.1s finished


0.9997909847775208

In [0]:
extra_trees.score(test_set_x_prepared,test_set_y)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


0.9675384416331634

We can fit to the training set at least.

### Extreme Boosting ###

In [0]:
!pip3 install xgboost

In [0]:
import xgboost as xgb


In [0]:
xgbmodel = xgb.XGBRegressor(n_jobs=-1, verboxity=2, random_state=42, n_estimators=100, objective="reg:squarederror", max_depth=20,)

In [0]:
xgbmodel.fit(train_set_x_prepared,train_set_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1, verboxity=2)

In [0]:
from sklearn.metrics import r2_score



In [0]:
r2_score(train_set_y, xgbmodel.predict(train_set_x_prepared))

0.9975928970862603

In [0]:
train_set_y

,maize_a_2010
140076,5778.3
233620,7638.7
321377,1568.3
125005,5132.8
157050,2242.6
...,...
149710,2614.2
332415,618.8
164446,953.7
184376,1051.1


## Hyperparameter tuning: search of parameter space ## 

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

### Random Forest (h2o) ###

In [0]:
#create small version of train set#

train_set_x_sample = train_set_x.sample(n=100, random_state=42)

In [0]:
train_set_y_sample = train_set_y.sample(n=100, random_state=42)

In [206]:
import h2o
h2o.init(nthreads = -1, max_mem_size = 12)

train = pd.DataFrame(train_set_x_sample).join(train_set_y_sample)
train = h2o.H2OFrame(train)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbzxrxu86
  JVM stdout: /tmp/tmpbzxrxu86/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpbzxrxu86/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_unknownUser_ysulr2
H2O cluster total nodes:,1
H2O cluster free memory:,12 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
x = train.col_names[:-1]    
y = train.col_names[-1]  

In [0]:
from h2o.grid.grid_search import H2OGridSearch


In [0]:
#hyperparameters
rf_parameters = {'max_depth': [200, 1000, 2000],
                'ntrees': [10, 20, 30],
                 'min_rows': [1, 100, 1000]}

rf_parameters2 = {'max_depth': list(np.random.randint(200,5000,10)),
                'ntrees': list(np.random.randint(10,20,10)),
                 }

# Search criteria


In [0]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

rf_h2o = H2ORandomForestEstimator(model_id='rf_h2o', seed=1, categorical_encoding='enum', nfolds=4, keep_cross_validation_models=True)

In [0]:
rf_h2o_grid = H2OGridSearch(model=rf_h2o,
                          grid_id='rf_h2o_grid',
                          hyper_params=rf_parameters2,
                         search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 20}

                          )

In [215]:
rf_h2o_grid.train(x=x, y=y, training_frame=train)

drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [216]:
rf_h2o_grid.show()

     max_depth ntrees             model_ids  residual_deviance
0         3928     10  rf_h2o_grid_model_15  4391705.047062643
1         4063     10  rf_h2o_grid_model_27  4391705.047062643
2         2690     10  rf_h2o_grid_model_28  4391705.047062643
3         2242     10  rf_h2o_grid_model_35  4391705.047062643
4         3106     10  rf_h2o_grid_model_39  4391705.047062643
5         3004     10   rf_h2o_grid_model_8  4391705.047062643
6         3106     19  rf_h2o_grid_model_12  4453786.879002184
7         2830     19  rf_h2o_grid_model_18  4453786.879002184
8         3004     19   rf_h2o_grid_model_2  4453786.879002184
9         4063     19  rf_h2o_grid_model_23  4453786.879002184
10         845     18  rf_h2o_grid_model_20  4567539.050942978
11        2690     18  rf_h2o_grid_model_29  4567539.050942978
12        2830     18  rf_h2o_grid_model_36  4567539.050942978
13        3106     18  rf_h2o_grid_model_38  4567539.050942978
14        2472     18   rf_h2o_grid_model_5  4567539.05

In [0]:
gbm_gridperf2 = rf_h2o_grid.get_grid(sort_by='r2', decreasing=True)


In [200]:
gbm_gridperf2.get_grid()

     max_depth ntrees             model_ids                  r2
0         3851     11  rf_h2o_grid_model_21   0.699807056511188
1         1610     11  rf_h2o_grid_model_22   0.699807056511188
2         2345     11  rf_h2o_grid_model_25   0.699807056511188
3         1301     11  rf_h2o_grid_model_29   0.699807056511188
4         1648     10   rf_h2o_grid_model_3  0.6845093334022347
5         1986     10   rf_h2o_grid_model_4  0.6845093334022347
6         1610     19  rf_h2o_grid_model_16  0.6800495078146043
7         1301     19  rf_h2o_grid_model_18  0.6800495078146043
8         2464     19  rf_h2o_grid_model_23  0.6800495078146043
9         4940     19  rf_h2o_grid_model_26  0.6800495078146043
10        4163     19  rf_h2o_grid_model_30  0.6800495078146043
11        2464     18  rf_h2o_grid_model_19   0.671877795878588
12        2345     18  rf_h2o_grid_model_24   0.671877795878588
13        3586     18  rf_h2o_grid_model_27   0.671877795878588
14        1986     13   rf_h2o_grid_mode

In [152]:
len(gbm_gridperf2.models)

12

In [0]:
rf_h2o.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [0]:
#rf_h2o.cross_validation_models()

In [224]:
h2o.shutdown()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  """Entry point for launching an IPython kernel.


H2O session _sid_b0ca closed.


#### sklearn ####

In [0]:
parameter_grid = {

  'n_estimators': sp_randint(8,40),
  'max_depth':  sp_randint(800,10000),
  'min_samples_split': uniform(0.0001,0.8)

}

In [0]:
from sklearn.ensemble import RandomForestRegressor

rnd_search = RandomForestRegressor()

random_search_rf = RandomizedSearchCV(rnd_search, parameter_grid, cv=5, n_jobs=-1, random_state=42, verbose=10)

In [230]:
random_search_rf.fit(train_set_x_prepared,train_set_y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 18.7min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [0]:
#random_search_rf.cv_results_

In [0]:
result = pd.concat([pd.DataFrame(random_search_rf.cv_results_["params"]),pd.DataFrame(random_search_rf.cv_results_["mean_test_score"], columns=["Score"])],axis=1)


In [0]:
result.sort_values(by="Score", ascending=False, inplace=True)

In [239]:
result

#clearly we need a smaller min_samples_split

,max_depth,min_samples_split,n_estimators,Score
5,7749,0.000723,35,0.952434
4,9122,0.016568,9,0.912773
2,7065,0.124896,18,0.869070
9,2847,0.233816,23,0.736472
7,7196,0.345656,24,0.722565
1,5991,0.479027,14,0.658738
6,7220,0.494085,33,0.658445
8,9466,0.489582,17,0.657863
3,6378,0.693041,11,-0.000022
0,8070,0.637334,22,-0.000023


In [0]:
parameter_grid = {

  'n_estimators': sp_randint(8,40),
  'max_depth':  sp_randint(800,10000),
  'min_samples_split': uniform(0,0.017)

}

from sklearn.ensemble import RandomForestRegressor

rnd_search = RandomForestRegressor()

random_search_rf2 = RandomizedSearchCV(rnd_search, parameter_grid, cv=5, n_jobs=-1, random_state=42, verbose=10)

In [0]:
random_search_rf2.fit(train_set_x_prepared,train_set_y)

result2 = pd.concat([pd.DataFrame(random_search_rf2.cv_results_["params"]),pd.DataFrame(random_search_rf2.cv_results_["mean_test_score"], columns=["Score"])],axis=1)
result2.sort_values(by="Score", ascending=False, inplace=True)

print(result2)

### Extra Trees ###

In [0]:
parameter_grid = {

  'n_estimators': sp_randint(1,50),
  'max_depth':  sp_randint(1,40),
  'min_samples_split': uniform(0.01,0.199)

}

In [0]:
from sklearn.ensemble import ExtraTreesRegressor

extra_trees_search = ExtraTreesRegressor()

In [0]:
random_search = RandomizedSearchCV(extra_trees_search, parameter_grid, cv=5, n_jobs=-1, random_state=42)

In [0]:
random_search.fit(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False...
                   para

In [0]:
random_search.cv_results_

{'mean_fit_time': array([ 72.47455616, 205.2609889 ,  70.10993247, 229.11446571,
         10.60546746, 180.70490422,  43.5544241 , 138.95962658,
        243.65751877,  62.53788862]),
 'mean_score_time': array([0.13923707, 0.3873415 , 0.17662063, 0.42622657, 0.05305338,
        0.2810226 , 0.09270654, 0.21287332, 0.28956146, 0.09422545]),
 'mean_test_score': array([0.86505246, 0.87791248, 0.90934645, 0.89583211, 0.84559105,
        0.86720167, 0.60017713, 0.89855338, 0.89839234, 0.85528205]),
 'param_max_depth': masked_array(data=[39, 8, 19, 24, 24, 24, 2, 33, 25, 28],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[0.16851205438518635, 0.12913303835521028,
                    0.029895008247782573, 0.07640801361666534,
                    0.13952680611682175, 0.17565608551928394,
                    0.04618316847421303, 0.07054

In [0]:
random_search.best_estimator_

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=19, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=0.029895008247782573,
                    min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=None,
                    oob_score=False, random_state=None, verbose=0,
                    warm_start=False)

In [0]:
random_search.best_score_

0.909346448145319

In [0]:
random_search.scorer_

<function sklearn.metrics._scorer._passthrough_scorer>

In [0]:
extra_trees3 = ExtraTreesRegressor(n_jobs=-1, verbose=1, n_estimators=10)
extra_trees3 = extra_trees.fit(train_set_x_prepared, train_set_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


In [0]:
extra_trees3.score(test_set_x_prepared,test_set_y)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.3s finished


0.9678790055348738

In [0]:
extra_trees3

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=10, n_jobs=-1, oob_score=False,
                    random_state=None, verbose=1, warm_start=False)

In [0]:
parameter_grid = {

  'n_estimators': sp_randint(9,30),
  'max_depth':  sp_randint(9,45),
  'min_samples_split': uniform(0.02,0.08)

}

from sklearn.ensemble import ExtraTreesRegressor

extra_trees_search2 = ExtraTreesRegressor()

random_search2 = RandomizedSearchCV(extra_trees_search2, parameter_grid, cv=5, n_jobs=-1, random_state=42, n_iter=40)

In [0]:
random_search2.fit(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                                 criterion='mse',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False...
                   para

In [0]:
random_search2.cv_results_

{'mean_fit_time': array([ 84.32273269, 148.47925954, 153.79277172,  55.99990792,
        110.24842634, 161.64903345, 101.58182349,  86.62750764,
         94.58214812,  62.96685677, 118.17540379,  95.687854  ,
         53.96593237,  74.13843327, 127.88408675,  74.76687455,
         63.06085238, 121.82361832,  81.41356611, 109.53628197,
         87.55894737, 138.53733177, 104.67712998,  39.94230399,
         37.11128922,  97.85231433,  49.55776749,  83.43548694,
         72.72540288, 129.27159901,  64.36589737,  56.9669538 ,
        113.62269115,  56.3964992 , 107.79695959, 108.51998024,
        120.34500971, 155.27775283,  61.24376035,  93.25170255]),
 'mean_score_time': array([0.16715827, 0.27495127, 0.30496855, 0.12990866, 0.21879888,
        0.29295297, 0.19992509, 0.18271151, 0.1874197 , 0.13152809,
        0.23628788, 0.19441452, 0.12681284, 0.16223497, 0.24922543,
        0.16417675, 0.14648447, 0.25047307, 0.17102652, 0.22177305,
        0.18186059, 0.260218  , 0.20654087, 0.0948

In [0]:
random_search2.best_score_

NameError: ignored

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
parameter_grid = {

  'n_estimators': [5, 10, 15] ,
  'max_depth': [20, 100, 1000],
  'min_samples_split': [0.0001, 0.02, 0.029]

}

from sklearn.ensemble import ExtraTreesRegressor

extra_trees_search3 = ExtraTreesRegressor()

random_search3 = GridSearchCV(extra_trees_search3, parameter_grid, cv=5, n_jobs=-1)

In [0]:
random_search3.fit(train_set_x_prepared,train_set_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0,
                                           criterion='mse', max_depth=None,
                                           max_features='auto',
                                           max_leaf_nodes=None,
                                           max_samples=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=100, n_jobs=None,
                                           oob_score=False, random_state=None,
                                           verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param

In [0]:
random_search3.cv_results_

{'mean_fit_time': array([ 34.28231077,  68.9629395 , 104.5328629 ,  28.32093906,
         52.2639636 ,  83.54023104,  26.6851954 ,  54.81241307,
         81.95760574,  37.08497033,  73.80888829, 108.7681221 ,
         25.44611597,  53.38589854,  77.25632534,  26.40457644,
         52.15934062,  78.49350448,  36.93231158,  74.08780155,
        108.9596416 ,  26.66802902,  56.20716553,  82.1959415 ,
         27.21377711,  52.96828947,  75.16814995]),
 'mean_score_time': array([0.1204958 , 0.22945061, 0.32889905, 0.07954755, 0.12660527,
        0.18292122, 0.07507524, 0.12489147, 0.17010279, 0.17162318,
        0.30061865, 0.4392848 , 0.08324237, 0.13203239, 0.18280563,
        0.07824707, 0.12936006, 0.175806  , 0.17596555, 0.30059633,
        0.4516017 , 0.07989416, 0.13577571, 0.18510046, 0.07961636,
        0.12783799, 0.15952253]),
 'mean_test_score': array([0.95816917, 0.95927646, 0.9602826 , 0.9152354 , 0.91591678,
        0.9184584 , 0.90999768, 0.91278388, 0.91289688, 0.96508428,

In [0]:
random_search3.best_score_

0.9677847951160079

In [0]:
random_search3.best_estimator_

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=1000, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=0.0001, min_weight_fraction_leaf=0.0,
                    n_estimators=15, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

### Extreme Boosting ###